In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Dataset being reviewed contains data about fights held by MMA promotion UFC. Each row represents a separate fight, including match-specific information such as winner and finish details, and also data about fighters: their rank, age, average takedown accuracy and so on.

Starting with loading and reviewing the dataset.

In [167]:
df = pd.read_csv('./data/ufc-master.csv')

In [168]:
df.shape

(4307, 113)

In [169]:
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs
0,Robert Whittaker,Darren Till,-130,107,76.923077,107.000000,7/25/2020,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,5.0,5:00,1500.0
1,Mauricio Rua,Rogerio Nogueira,-190,150,52.631579,150.000000,7/25/2020,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,Red,S-DEC,NaN,3.0,5:00,900.0
2,Fabricio Werdum,Alexander Gustafsson,260,-335,260.000000,29.850746,7/25/2020,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,Blue,SUB,Armbar,1.0,2:30,150.0
3,Carla Esparza,Marina Rodriguez,145,-182,145.000000,54.945055,7/25/2020,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,Red,S-DEC,NaN,3.0,5:00,900.0
4,Paul Craig,Gadzhimurad Antigulov,-137,110,72.992701,110.000000,7/25/2020,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,neither,SUB,Triangle Choke,1.0,2:06,126.0


I'm gonna start with closer reviewing data. First let's separate numeric and categorical variables, check amount of null values.
B_odds and R_odds values are strings, will convert those to int.

In [170]:
df[['B_odds', 'R_odds']] = df[['B_odds', 'R_odds']].astype(int)

In [171]:
cat_vars = df.select_dtypes(include=['object']).columns
num_vars = df.select_dtypes(exclude=['object']).columns

In [172]:
print(len(num_vars))
print(df[num_vars].isna().sum().to_string())

99
R_odds                             0
B_odds                             0
R_ev                               0
B_ev                               0
title_bout                         0
no_of_rounds                       0
B_current_lose_streak              0
B_current_win_streak               0
B_draw                             0
B_avg_SIG_STR_landed             930
B_avg_SIG_STR_pct                765
B_avg_SUB_ATT                    832
B_avg_TD_landed                  833
B_avg_TD_pct                     842
B_longest_win_streak               0
B_losses                           0
B_total_rounds_fought              0
B_total_title_bouts                0
B_win_by_Decision_Majority         0
B_win_by_Decision_Split            0
B_win_by_Decision_Unanimous        0
B_win_by_KO/TKO                    0
B_win_by_Submission                0
B_win_by_TKO_Doctor_Stoppage       0
B_wins                             0
B_Height_cms                       0
B_Reach_cms                        

99 numeric values in total.

Some fighters used to fight in different weights than the current one, thus those also have their former ranks available. In order to keep it simple, I will drop those columns and only keep fight weight rank.

Match weight rank is also not available for all the fighters. Only top 15 are ranked, the lower the rank the better. We can not replace nan value with 0 here, so I will try to set it to a bigger value (20) and also add new column indicating that a fighter belongs to top 15.

In [173]:
df[['B_match_weightclass_rank', 'R_match_weightclass_rank']] = df[['B_match_weightclass_rank', 'R_match_weightclass_rank']].fillna(20)

In [174]:
df['B_is_in_top15'] = [1 if x < 20 else 0 for x in df['B_match_weightclass_rank']]
df['R_is_in_top15'] = [1 if x < 20 else 0 for x in df['R_match_weightclass_rank']]

Some rows have "finish_round" and "total_fight_time_secs" empty, which is weird. These rows also do not have any details about how match finished. I guess I will put zeroes here.

What we have left with NaNs are submissions and takedowns data which do not happen in all off the fights. Seems legit to replace missing values with 0 here.

In [175]:
df[num_vars] = df[num_vars].fillna(0)

Moving to categorica data. Firt I want to check amount of missing values as well.

In [176]:
df[cat_vars].isna().sum()

R_fighter               0
B_fighter               0
date                    0
location                0
country                 0
Winner                  0
weight_class            0
gender                  0
B_Stance                0
R_Stance                0
better_rank             0
finish                225
finish_details       2311
finish_round_time     225
dtype: int64

Same 225 missing values in "finish" and "finish_round_time". "finish_details" missing values seem to be quite legit. Most of the fights end with decision, and there is nothing more to say about it. 
I will fill missing values with constant string 'undefined' for "finish" and "finish_details". 

"finish_round_time" intuitively does not seem to belong to categorical. I want to convert it to single number (seconds) and use ans numeric variable.

In [177]:
df[['finish', 'finish_details']] = df[['finish', 'finish_details']].fillna('undefined')

In [178]:
def min_to_sec(s):
    ss = str(s).split(':')
    return int(ss[0])*60 + int(ss[1])

df['finish_round_time'] = df['finish_round_time'].fillna('0:0')
df['finish_round_seconds'] = df['finish_round_time'].apply(min_to_sec)

I am not going to remove anything from the original dataset, but rather continie building 2 Index entities (num_vars and cat_vars) which I will use further during model building. 

It's pretty clear that fighters' names hardly influence results, so I will not include those. Same applies to location where the fights is held. 

I also want to convert "date" columns to datetime format for possible future usage during EDA or some auxilary manipulations. This column will not participate in model building.

In [179]:
df['date'] = pd.to_datetime(df['date'], yearfirst=True, format='%m/%d/%Y')

"gender" clearly can be boolean variable. I will also turn 'Winner' and 'better_rank' into boolean. 1 will be Blue, 0 - Red.

In [180]:
df['gender_bool'] = [1 if x == 'MALE' else 0 for x in df['gender']]
df['winner_bool'] = [1 if x == 'Blue' else 0 for x in df['Winner']]
df['better_rank_bool'] = [1 if x == 'Blue' else 0 for x in df['better_rank']]

In [184]:
cat_vars = pd.Index(['country', 'weight_class', 'B_Stance', 'R_Stance', 'finish', 'finish_details'])

In [216]:
num_vars = pd.Index(['R_odds', 'B_odds', 'R_ev', 'B_ev', 'title_bout', 'no_of_rounds',
       'B_current_lose_streak', 'B_current_win_streak', 'B_draw',
       'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Height_cms', 'B_Reach_cms',
       'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak',
       'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT',
       'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses',
       'R_total_rounds_fought', 'R_total_title_bouts',
       'R_win_by_Decision_Majority', 'R_win_by_Decision_Split',
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif', 'win_streak_dif',
       'longest_win_streak_dif', 'win_dif', 'loss_dif', 'total_round_dif',
       'total_title_bout_dif', 'ko_dif', 'sub_dif', 'height_dif', 'reach_dif',
       'age_dif', 'sig_str_dif', 'avg_sub_att_dif', 'avg_td_dif',
       'empty_arena', 'finish_round', 'total_fight_time_secs', 'B_is_in_top15', 
        'R_is_in_top15', 'finish_round_seconds', 'gender_bool', 
        'better_rank_bool'])

In [185]:
df[cat_vars].head()

,country,weight_class,B_Stance,R_Stance,finish,finish_details
0,United Arab Emirates,Middleweight,Southpaw,Orthodox,U-DEC,undefined
1,United Arab Emirates,Light Heavyweight,Southpaw,Orthodox,S-DEC,undefined
2,United Arab Emirates,Heavyweight,Orthodox,Orthodox,SUB,Armbar
3,United Arab Emirates,Women's Strawweight,Orthodox,Orthodox,S-DEC,undefined
4,United Arab Emirates,Light Heavyweight,Orthodox,Orthodox,SUB,Triangle Choke


In [186]:
for ind in cat_vars:
    print(ind)
    print(df[ind].value_counts())

country
 USA                     2450
 Brazil                   400
 Canada                   337
 United Kingdom           165
 Australia                160
USA                       100
 Sweden                    72
 Mexico                    70
 China                     61
 Germany                   54
 Japan                     53
United Arab Emirates       51
 Singapore                 45
 Russia                    36
 New Zealand               33
 United Arab Emirates      29
 Netherlands               25
 South Korea               24
 Poland                    23
 Ireland                   19
 Croatia                   13
 Czech Republic            13
 Uruguay                   13
 Denmark                   13
 Chile                     13
 Argentina                 12
 Philippines               12
Brazil                     11
Name: country, dtype: int64
weight_class
Lightweight              789
Welterweight             759
Middleweight             516
Featherweight           

Some country names have leading whitespace, need to clean this up.

In [187]:
df['country'] = df['country'].apply(lambda x: str(x).strip())

All columns seem to have reasonably low cardinality, so we can apply One Hot encoding (or dummy variables) when building the model.

## IT'S TIME  (in Bruce Buffer's voice)##

I will start my analysis trying to answer the following question: 

### What influences fight result the most?

Do I have to look for something complicated or is there a simple winning strategy?

Main idea of this whole analysis is to see if I can bet UFC fight results with some profit. What if I always keep the same strategy? Say, always bet the guy having more wins in past will win? I can hardly imagine myself betting for 10 years, so let's take 2019 and 2020 only.

In [199]:
df_19_20 = df[df['date']>='2019-01-01']
bet_win = [(x['B_ev']+100)*x['winner_bool'] if x['win_dif'] > 0 else (x['R_ev']+100)*(1-x['winner_bool']) for _, x in df_19_20.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

-4260.27 USD


Alright, going with this strategy I would lose 4k in less than 2 years. Does not look like a success story for me.

Maybe the yonger fighter?

In [200]:
bet_win = [(x['B_ev']+100)*x['winner_bool'] if x['age_dif'] < 0 else (x['R_ev']+100)*(1-x['winner_bool']) for _, x in df_19_20.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

-5243.01 USD


Having longer reach?

In [201]:
bet_win = [(x['B_ev']+100)*x['winner_bool'] if x['reach_dif'] > 0 else (x['R_ev']+100)*(1-x['winner_bool']) for _, x in df_19_20.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

-2174.17 USD


Maybe the one showing greater average amount of significant strikes?

In [202]:
bet_win = [(x['B_ev']+100)*x['winner_bool'] if x['sig_str_dif'] > 0 else (x['R_ev']+100)*(1-x['winner_bool']) for _, x in df_19_20.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

3615.10 USD


Finally something! Won't make me a millionaire, but better than losing money.

But what does affect the match result in the end? Instead of making guesses let me build the model!

Task can be treated as binary classification. In the dataframe we always have data available in fixed order: first it's fighter in the blue corner, then - in the right. Target varible 'winner_bool' is 1 if Blue wins, 0 - if Right. Thus B_ columns can be treated as 'fighter data', and R_ columns - as 'opponent data' and we want to know whether the fighter wins or not.

Some of the variables should be removed before building the model. If we want to predict result of the match, we can not use information about whether the match finished with decision or submission and in which round it finished. Numeric values also should be stardatized, categorical values should be encoded.

In [249]:
cat_vars = pd.Index(['weight_class', 'B_Stance', 'R_Stance'])
num_vars_no_finish = num_vars.drop(['finish_round', 'total_fight_time_secs', 'finish_round_seconds'])

In [207]:
X = pd.get_dummies(df[cat_vars])
X = pd.concat([df[num_vars_no_finish], X], axis=1)
y = df['winner_bool']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [208]:
scaler = MinMaxScaler()
#X_train.loc[:, num_vars] = scaler.fit_transform(X_train[num_vars])
#X_test.loc[:, num_vars] = scaler.transform(X_test[num_vars])
X = scaler.fit_transform(X)

We can now build simple Logistic Regression model. Instead of splitting into train and test sets I will use k-fold cross-validation strategy with 3 folds.

In [209]:
log_reg = LogisticRegression()
#log_reg.fit(X_train, y_train)
#y_pred = log_reg.predict(X_test)
#print(accuracy_score(y_test, y_pred))
cross_val_score(log_reg, X, y, cv=3).mean()

0.6468558617142081

This does not seem to give too good accuracy. Regression is not the only model we can build. I also want to try Random Forest and Gradient Boosting.

In [210]:
gb_clf = GradientBoostingClassifier()
#gb_clf.fit(X_train, y_train)
#y_pred = gb_clf.predict(X_test)
#print(accuracy_score(y_test, y_pred))
cross_val_score(gb_clf, X, y, cv=3).mean()

0.6466230883956272

In [211]:
rf_clf = RandomForestClassifier()
#rf_clf.fit(X_train, y_train)
#y_pred = rf_clf.predict(X_test)
#print(accuracy_score(y_test, y_pred))
cross_val_score(rf_clf, X, y, cv=3).mean()

0.654747346319464

I tried tuning some hyperparameters here, which made no significant improvements, so I will stick to default values. Looks like without additional precessing and maybe feature engineering this is the maximum we can get.

Even so, I want to check which columns make the most contribution.

In [257]:
def train_model(df, cat_vars, num_vars, model):
    """
    encodes cat_vars, scales num_vars, performs train-test split, trains model.
    return model coefficients and dataframe columns
    """
    X = pd.get_dummies(df[cat_vars])
    X = pd.concat([df[num_vars], X], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    scaler = MinMaxScaler()
    X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
    X_test[num_vars] = scaler.transform(X_test[num_vars])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    return model.coef_[0], X_train.columns

In [260]:
def get_coef(model_coef, df_columns):
    """
    builds dataframe containing model coefficients for dataframe columns
    """
    coef_df = pd.DataFrame()
    coef_df['column'] = df_columns
    coef_df['coef'] = model_coef
    coef_df['abs_coefs'] = np.abs(model_coef)
    coef_df = coef_df.sort_values('abs_coefs', ascending=False)
    return coef_df

In [261]:
log_reg_coef, X_columns = train_model(df, cat_vars, num_vars_no_finish, LogisticRegression())
coef_df = get_coef(log_reg_coef, X_columns)
coef_df.head(10)

0.6581593194122196


,column,coef,abs_coefs
3,B_ev,-2.834850,2.834850
2,R_ev,1.850180,1.850180
0,R_odds,1.045692,1.045692
27,B_Weight_lbs,1.039969,1.039969
41,R_win_by_Decision_Split,0.946774,0.946774
34,R_avg_TD_landed,-0.863705,0.863705
20,B_win_by_Decision_Unanimous,0.852874,0.852874
1,B_odds,-0.832704,0.832704
7,B_current_win_streak,0.721106,0.721106
30,R_draw,-0.682088,0.682088


What a surprise (no)! Most meaningful variables turn out to be the ones related to odds. 

Interenting enough. If we set logistic regression to use L1 penalty which eliminates features rather that optimizes coefficients, it removes "_odd" features.

In [262]:
log_reg_coef, X_columns = train_model(df, cat_vars, num_vars_no_finish, 
                           LogisticRegression(solver='liblinear', penalty='l1'))
coef_df = get_coef(log_reg_coef, X_columns)
coef_df.head(10)

0.6697602474864656


,column,coef,abs_coefs
3,B_ev,-4.744729,4.744729
2,R_ev,2.361859,2.361859
34,R_avg_TD_landed,-1.018101,1.018101
41,R_win_by_Decision_Split,0.867094,0.867094
27,B_Weight_lbs,0.863464,0.863464
14,B_longest_win_streak,-0.811024,0.811024
20,B_win_by_Decision_Unanimous,0.647807,0.647807
7,B_current_win_streak,0.616956,0.616956
51,B_age,-0.602495,0.602495
30,R_draw,-0.544115,0.544115


Which of course makes sense. There is strong connection between ev and odd features. It we calculate correlation (I use Spearman correlation here as it is reasonable to assume relation is monotonous, not necessarily linear), we can clearly see its absolute value is very close to 1.

In [264]:
df[['B_ev', 'R_ev', 'B_odds', 'R_odds']].corr(method='spearman')

,B_ev,R_ev,B_odds,R_odds
B_ev,1.000000,-0.998422,0.999973,-0.998386
R_ev,-0.998422,1.000000,-0.998391,0.999963
B_odds,0.999973,-0.998391,1.000000,-0.998415
R_odds,-0.998386,0.999963,-0.998415,1.000000


Assuming strong correlation between all 4, we can only leave one of these in the model.

In [265]:
num_vars_no_ev = num_vars_no_finish.drop(['R_odds', 'B_ev', 'R_ev'])
log_reg_coef, X_columns = train_model(df, cat_vars, num_vars_no_ev, LogisticRegression())
coef_df = get_coef(log_reg_coef, X_columns)
coef_df.head(10)

0.6488785769528229


,column,coef,abs_coefs
0,B_odds,-5.210548,5.210548
24,B_Weight_lbs,1.054625,1.054625
38,R_win_by_Decision_Split,0.999226,0.999226
17,B_win_by_Decision_Unanimous,0.876141,0.876141
31,R_avg_TD_landed,-0.875505,0.875505
4,B_current_win_streak,0.688672,0.688672
11,B_longest_win_streak,-0.674845,0.674845
27,R_draw,-0.656884,0.656884
63,avg_td_dif,0.642496,0.642496
19,B_win_by_Submission,0.637838,0.637838


Accuracy is almost the same. If we remove the odds data completely, here is what we get.

In [267]:
num_vars_no_odds = num_vars_no_ev.drop(['B_odds'])
log_reg_coef, X_columns = train_model(df, cat_vars, num_vars_no_odds, LogisticRegression())
coef_df = get_coef(log_reg_coef, X_columns)
coef_df.head(10)

0.6078886310904872


,column,coef,abs_coefs
33,R_losses,1.291261,1.291261
37,R_win_by_Decision_Split,1.220752,1.220752
30,R_avg_TD_landed,-1.190440,1.190440
23,B_Weight_lbs,1.157398,1.157398
47,B_age,-1.098218,1.098218
3,B_current_win_streak,0.885995,0.885995
45,R_Weight_lbs,-0.881755,0.881755
16,B_win_by_Decision_Unanimous,0.870295,0.870295
46,R_age,0.855394,0.855394
22,B_Reach_cms,0.772402,0.772402


Low enough accuracy goes even lower.

Conclusion is simple here. Odds do not come from nowhere and you probably want to take those into consideration. Still, let me check, how much I can earn always betting the fighter with better odds.

In [268]:
bet_win = [(x['B_ev']+100)*x['winner_bool'] if x['B_odds'] < 0 else (x['R_ev']+100)*(1-x['winner_bool']) for _, x in df_19_20.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

-3937.50 USD


Let me change train-test split a bit. I want my model to fit on data prior to 2019 and use it's predictions for later fights. I will check the accuracy and amount of money I can earn on bets.

In [294]:
X = pd.get_dummies(df[cat_vars])
X = pd.concat([df[num_vars_no_finish], X], axis=1)
X['winner_bool'] = df['winner_bool']
X_train = X[df['date']<'2019-01-01']
X_test = X[df['date']>='2019-01-01']
y_train = X_train['winner_bool']
X_train = X_train.drop(['winner_bool'], axis=1)
y_test = X_test['winner_bool']
X_test = X_test.drop(['winner_bool'], axis=1)
scaler = MinMaxScaler()
X_train[num_vars_no_finish] = scaler.fit_transform(X_train[num_vars_no_finish])
X_test[['B_ev_orig', 'R_ev_orig']] = X_test[['B_ev', 'R_ev']] 
X_test[num_vars_no_finish] = scaler.transform(X_test[num_vars_no_finish])
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test.drop(['B_ev_orig', 'R_ev_orig'], axis=1))
print(accuracy_score(y_test, y_pred))

0.6383265856950068


In [296]:
X_test['winner_pred'] = y_pred
X_test['winner_bool'] = y_test

In [299]:
bet_win = [(x['B_ev_orig']+100)*x['winner_bool'] if x['winner_pred'] else (x['R_ev_orig']+100)*(1-x['winner_bool']) for _, x in X_test.iterrows()]
print("{:.2f} USD".format(np.sum(bet_win)-100*df_19_20.shape[0]))

-932.49 USD


Surprised? Bad result is also a result. I am quite sure we CAN build good enough model in the end with some feature selection, feature engineering and hyperparameter tuning, but there is no simple one minute solution for this. What you always can do is enjoy a spectacular fight.

Speaking of which. I am not a fan of bouts ending with decision. I love knockounts and submission. Which leads me to the next question.

### Is there anything special about fights not ending with decision?